# Calculation of the sensitivity

In [1]:
# Generic libraries
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import signal as sps
from os.path import join as osj
import pickle
import wfdb
import os
import copy
import random

# Differential privacy libraries
from diffprivlib import mechanisms
from diffprivlib import models
from diffprivlib import tools
from diffprivlib.accountant import BudgetAccountant
from diffprivlib.utils import check_random_state
from diffprivlib.mechanisms import Laplace, LaplaceBoundedNoise, LaplaceBoundedDomain
from diffprivlib.mechanisms import DPMechanism

In [2]:
def get_patient_ids():
    # patient_ids = pd.read_csv(RECORDS, delimiter="\n", header=None).to_numpy().reshape(-1)
    patient_ids = pd.read_csv(osj("..", "files", "patient_ids.csv"), header=None).to_numpy().reshape(-1)
    return patient_ids

def get_valid_patients():
    # valid_patient_ids = pd.read_csv(RECORDS, delimiter="\n", header=None).to_numpy().reshape(-1)
    valid_patient_ids = pd.read_csv(osj("..", "files", "valid_patients.csv"), header=None).to_numpy().reshape(-1)
    return valid_patient_ids

def get_ecg_signals(patient_ids):
    """
    The MIT-BIH data was generated with 2 leads. Per patient this function reads the ecg data per lead.
    """
    lead0 = {}
    lead1 = {}
    for id_ in patient_ids:
        signals, _ = wfdb.io.rdsamp(osj(ROOT, str(id_)))
        lead0[id_] = signals[:, 0]
        lead1[id_] = signals[:, 1]
    return lead0, lead1

def get_ecg_info(patient_ids):
    """
    The MIT-BIH data additionally contains ecg info, providing additional information, such as the age, sex, gender, and comments.
    """
    info = {}
    for id_ in patient_ids:
        _, info_ = wfdb.io.rdsamp(osj(ROOT, str(id_)))
        info[id_] = info_["comments"][0]
    return info

def get_all_beat_labels(patient_ids):
    """
    Getting the unique set of labels that are present in the MIT data.
    """
    all_labels = []
    for id_ in patient_ids:
        annotation = wfdb.rdann(osj(ROOT, str(id_)), extension='atr')
        labels = np.unique(annotation.symbol)
        all_labels.extend(labels)
    return np.unique(all_labels)

def get_rpeaks_and_labels(patient_ids):
    """
    Getting the ids of the r-peaks and the corresponding annotated label.
    """
    rpeaks = {}
    labels = {}
    for id_ in patient_ids:
        annotation = wfdb.rdann(osj(ROOT, str(id_)), extension='atr')
        rpeaks[id_] = annotation.sample
        labels[id_] = np.array(annotation.symbol)
    return rpeaks, labels

def get_normal_beat_labels():
    """
    The MIT-BIH labels that are classified as healthy/normal. Check wfdb.Annotation documentation for description of labels.
    N: {N, L, R, e, j}. 
    """
    return np.array(["N", "L", "R", "e", "j"])

def get_abnormal_beat_labels():
    """
    The MIT-BIH labels that are classified as arrhythmia/abnormal. Check wfdb.Annotation documentation for description of labels.
    S: {S, A, J, a} - V: {V, E} - F: {F} - Q: {Q}
    """
    return np.array(["S", "A", "J", "a", "V", "E", "F", "Q"])

def get_global_sensitivity(valid_patients, lead0, labels):

    all_count = {patient: {"normal": 0, "abnormal": 0, "min": 0.0, "max": 0.0, "mean": 0.0} for patient in valid_patients}

    normal_labels = get_normal_beat_labels()
    abnormal_labels = get_abnormal_beat_labels()

    # get values per patient
    for patient in valid_patients:
        p_min_value = 0
        p_max_value = 0
        p_mean_value = 0

        count_normal = len(np.where(np.isin(labels[patient], normal_labels))[0])
        count_abnormal = len(np.where(np.isin(labels[patient], abnormal_labels))[0])
        p_mean_value = np.mean(lead0[patient])
        p_min_value = np.min(lead0[patient])
        p_max_value = np.max(lead0[patient])

        all_count[patient]['normal'] = count_normal
        all_count[patient]['abnormal'] = count_abnormal
        all_count[patient]['min'] = p_min_value
        all_count[patient]['max'] = p_max_value
        all_count[patient]['mean'] = p_mean_value

    # aggregate values while iteratively leaving one patient out
    all_count_agg = {patient: {"g_ratio": 0.0, "g_min": 0.0, "g_max": 0.0, "g_mean": 0.0} for patient in valid_patients}
    for patient_leavout in valid_patients:

        all_count_copy = copy.deepcopy(all_count)
        del all_count_copy[patient_leavout] 
        values = all_count_copy.values()

        normal = sum(patient["normal"] for patient in values)
        abnormal = sum(patient["abnormal"] for patient in values)
        ratio = abnormal / normal

        all_count_agg[patient_leavout]["g_ratio"] = ratio
        all_count_agg[patient_leavout]["g_min"]   = min(patient["min"] for patient in values)
        all_count_agg[patient_leavout]["g_max"]   = max(patient["max"] for patient in values)
        sum_mean                                  = sum(patient["mean"] for patient in values)
        all_count_agg[patient_leavout]["g_mean"]  = sum_mean / len(values)


    agg_values = all_count_agg.values()
    ratio_difference = max(patient["g_ratio"] for patient in agg_values)     - min(patient["g_ratio"] for patient in agg_values)
    min_difference   = abs(min(patient["g_min"] for patient in agg_values))  - abs(max(patient["g_min"] for patient in agg_values)) # abs(min) is bigger than abs(max)
    max_difference   = max(patient["g_max"] for patient in agg_values)       - min(patient["g_max"] for patient in agg_values)
    mean_difference  = abs(min(patient["g_mean"] for patient in agg_values)) - abs(max(patient["g_mean"] for patient in agg_values)) # abs(min) is bigger than abs(max)

    return ratio_difference

In [3]:

ROOT = osj("..", "physionet.org/files/mitdb/1.0.0")
RECORDS = osj(ROOT, "RECORDS")

patient_ids = get_valid_patients()
valid_patients = get_valid_patients()
lead0, lead1 = get_ecg_signals(patient_ids) # total 650.000 values in lead0 per patient
ecg_info = get_ecg_info(patient_ids)

# store patient meta data
df_patient = pd.DataFrame.from_dict(ecg_info, orient="index")
df_patient[['Age', 'Sex', 'value1', 'value2', 'x']] = df_patient[0].str.split(' ', expand=True)
df_patient.drop(columns=[0, 'value1', 'value2', 'x'], inplace=True)

rpeaks, labels = get_rpeaks_and_labels(patient_ids)

sensitivity = get_global_sensitivity(valid_patients, lead0, labels)

In [4]:
sensitivity

0.024576574702308618